<h1>Introduction</h1>
Telraam is one of 15 projects which have been brought to success with support from
`Smart Mobility Belgium`. 
<br>
The project started with a pilot study in Kessel-Lo. There, 100 residents carried out their own traffic counts using a specially developed low-cost sensor, Telraam, from their windows.

<h1>Telraam in Leuven</h1>

The `telraam` (called abacus in English) consists of a microcomputer (Raspberry Pi), and a low-resolution camera. The camera records the traffic, the microcomputer
immediately analyzes and processes the images for useful information. 
<br>
The data is transmitted wirelessly to a central database where these are continuously processed. The processed data of all counting frames are before
everyone freely available on www.telraam.net.
<br>
<br>
On the following <a target="_blank" href="https://www.telraam.net/en">map </a>, you can find the positioning of the cameras in Leuven

<h1>What object does the telraam count?</h1>
Telraam counts pedestrians, cyclists, cars and freight/heavy vehicles. This is done using images taken by the device camera and the analysis performed by the ‘Raspberry Pi’ (a small computer on which the device is based). The analysis simply uses the size and speed of the passing object.
<br>
There are 3 kinds of moving objects that are captured by the telraam: 
<ol>
    <li>a pedestrian
    <li>a cyclist 
    <li>motorized transport (vehicle)
</ol>
<br>
Motorized transport is encoded as either a car or a truck. With some caveats below:
<ol>
<li>Telraam does not count when it is dark.
<li>Telraam will count a tram as a truck.
<li>Vans can either be counted as trucks or cars, depending on their size.
<li>Buses are counted as trucks, because of their size.
</ol>

<h1>What does the telraam NOT count ? </h1>
<ol>
<li>Telraam does not recognise individual vehicles (number plates).
<li>Telraam does not recognise faces.
<li>Telraam does not count an EXACT number of vehicles over a very short period
<li>Telraam is not a speed camera.
<li>There is no 100% accuracy
<li>Telraam does not count when it is dark
</ol>

<h1>How does it work ? </h1>
Each Telraam unit uses a camera to monitor the street in its field of view (FOV). The images are being processed on-the-fly, meaning that there are no actual photos or videos stored or transferred from the units.
<br>
The telraam observes if something has passed in front of the house, but doesn’t know what kind of object it was. That is where the classifier needs to step in. The classifier decides if an observed object is a car (including also vans, buses, and trucks), a cyclists, or a pedestrian.
<br>
You can find more information on how the classifier works on the <a href="https://telraam.zendesk.com/hc/en-be/articles/360035184892-Potential-inaccuracies-in-Telraam">website</a> of telraam. (Detailed knowledge on this classification is not required in this datathon ). Just remember that two parameters are important for the telraam to identify an object as a car, bike or pedestrian: the <b>axisratio</b> and the <b>fullness</b>.
<br>
<br>
The global classifier itself is a lookup matrix that tells us for any axisratio – fullness parameter combination what kind of object we are dealing with. This makes classifying new observations extremely fast (as it is a simple lookup operation).
<br>
<br>
$$\text{Type Moving Object} = f(axisratio ,fullness) $$ 
<br>
The classifier is a global classifier. In other words, every camera of the telraam network is using the same method derived from a trained classifier on the first 3 camera's that were installed in Leuven.

<h1>What information is gather on the objects</h1>
Speeds are indicative. Measurements may differ by 10% from effective speeds.
<br>
The information retrieved by the different Telraams can be retrieve via an <a target = "_blank" href="https://documenter.getpostman.com/view/5886213/S11RLFhq/?version=latest" target="new">API</a>
<br>
<br>
If you <b>never worked with an API before</b>: Start reading <a target="_blank" href="https://www.dataquest.io/blog/python-api-tutorial/"><b>here</b></a>.

<h1> Datastructure</h1>
A Telraam gets data linked to a road segment. This is a unique relation.<br>

A Telraam is characterized by a unique serial number(`mac`), which when registered on the website, is linked to the unique road segment number (`segment_id`),

<h1>Retrieving information from a telraam </h1>
The information on the telraam is stored on a central database that can be retrieved using <b>API</b> calls.
<br>
As illustrated in the next cell, two python modules are required: <b>requests</b> and <b>json</b>
<br>
<br>
Online documentation is available on the following <a href = "https://documenter.getpostman.com/view/5886213/S11RLFhq/?version=latest">link</a>.

In [1]:
import requests
import json
import pandas as pd
from IPython.core.display import display, HTML

When you call the API with a `request` statement, you will get a response.
A response equal to 200 points to a success status and indicates that the request has succeeded.<br>
The answer to your request is often in a json format that can be sliced and diced for further information.

Find all the cameras (telraam) that are available :

In [2]:
answer = requests.get("https://telraam-api.net/v0/cameras")
nbr_cameras = len(answer.json()['cameras'][:])
print('Nbr of cameras:',nbr_cameras)

Nbr of cameras: 969


Each Camera has a number of characteristics. The 10th camera has the following attributes:

In [3]:
answer.json()['cameras'][9]

{'mac': '202481592771284',
 'user_id': '323',
 'segment_id': '279199',
 'direction': False,
 'status': 'active',
 'manual': False,
 'time_added': '2019-05-05T18:27:19.387Z',
 'last_data_package': '2020-02-15T13:46:06.804Z',
 'pedestrians_left': True,
 'pedestrians_right': True,
 'bikes_left': True,
 'bikes_right': True,
 'cars_left': True,
 'cars_right': True}

Each attribute has a particular meaning:
<ol>
<li>mac : unique identifier of the camera
<li>segment_id : Section of the street where the camera is active. This section has an id, and has properties that can be retrieved as well.
<li>direction': True,
<li>status: The telraam can be "active","no_active" or "problematic"
<li>manual : (not used),
<li>time_added:Timestamp when the telraam was added to the network,
<li>last_data_package: Timestamp of the retrieval of the most recent datapackage sent from the telraam,
<li>pedestrians_left: False
<li>pedestrians_right: True
<li>bikes_left: True
<li>bikes_right: True
<li>cars_left: True
<li>cars_right: True
</ol>

<h1>What is the segment ?</h1>


A segment is a section of a street where a telraam is active. Each segment has an id and it is possible to have several telraam devices active on an identical segment

In [4]:
segments = requests.get("https://telraam-api.net/v0/segments/active")
print(segments.status_code)

200


The API returns code 200 which points to a correct call

In [5]:
nbr_segments = len(segments.json()['features'][:])
print('Nbr of segments:',nbr_segments)

Nbr of segments: 817


Similar to the telraam, we can investigate the different attributes of each segment.
As an example we take the first segment in the list

In [6]:
segments.json()['features'][0].keys()

dict_keys(['type', 'geometry', 'properties'])

Each segment has properties and a geometry

In [7]:
segments.json()['features'][0]['geometry']

{'type': 'MultiLineString',
 'coordinates': [[[5.248800931878607, 50.958120376295234],
   [5.248909755505087, 50.95811997291265],
   [5.2497739115567805, 50.95811676630803],
   [5.249987769051002, 50.95811992722135],
   [5.250143365286705, 50.95809817266468],
   [5.250167188129611, 50.95809762914239],
   [5.250498571750504, 50.95809007448103]]]}

The coordinates of a segment form a line along the road that forms the segment.
<br>
<br>
The properties are given by :

In [8]:
segments.json()['features'][0]['properties']

{'id': 61753,
 'speed': 50,
 'oneway': False,
 'road_type': '',
 'road_speed': '',
 'pedestrian': 80,
 'bike': 85,
 'car': 5527,
 'lorry': 0,
 'last_data_package': None,
 'pedestrian_avg': 0,
 'bike_avg': 33,
 'car_avg': 400,
 'lorry_avg': 0,
 'typical_data': [{'hour': '05:00:00',
   'pedestrian': 0,
   'bike': 0,
   'car': 34,
   'lorry': 0},
  {'hour': '06:00:00', 'pedestrian': 0, 'bike': 2, 'car': 58, 'lorry': 0},
  {'hour': '07:00:00', 'pedestrian': 1, 'bike': 5, 'car': 107, 'lorry': 0},
  {'hour': '08:00:00', 'pedestrian': 0, 'bike': 6, 'car': 258, 'lorry': 0},
  {'hour': '09:00:00', 'pedestrian': 0, 'bike': 7, 'car': 162, 'lorry': 0},
  {'hour': '10:00:00', 'pedestrian': 1, 'bike': 7, 'car': 178, 'lorry': 0},
  {'hour': '11:00:00', 'pedestrian': 1, 'bike': 7, 'car': 212, 'lorry': 0},
  {'hour': '12:00:00', 'pedestrian': 1, 'bike': 10, 'car': 195, 'lorry': 0},
  {'hour': '13:00:00', 'pedestrian': 1, 'bike': 5, 'car': 196, 'lorry': 0},
  {'hour': '14:00:00', 'pedestrian': 0, 'bike'

<b> Explanation of the different attributes </b>
<ul>
<li>id: id of the segment
<li>speed: not used
<li>oneway: not used
<li>road_type: not used
<li>road_speed: not used
<li>pedestrian: Nbr of pedestrians counted during the last hour
<li>bike: Nbr of bikes counted during the last hour
<li>car: Nbr of cars counted during the last hour
<li>lorry: Nbr of lorries (vans) counted during the last hour,
<li>last_data_package: Timestamp of information received by a Telraam on this segment
<li>pedestrian_avg: Moving average
<li>bike_avg: Moving average
<li>car_avg: Moving average
<li>lorry_avg:Moving average 
</ul>
The attribute "typical_data" shows what data to expect on a particular time of the day

<ul>
    
   </ul>

In [9]:
df.mean()

NameError: name 'df' is not defined

In [ ]:
segment = '349811' 
url = 'https://telraam-api.net/v0/cameras/segment/'
new_url = url+segment
segment_info = requests.get(new_url)
print(segment_info.status_code)
segment_info.json()

Get Json information on the segments (GEOJSON Format)

In [ ]:
url = 'https://telraam-api.net/v0/segments/id/'
segment_id = '349811' 
new_url = url+segment_id
segment_info = requests.get(new_url)
print(segment_info.status_code)
segment_info.json()